In [64]:
import pandas as pd
import numpy as np
import openpyxl
import sys
import os
from pathlib import Path
import datetime

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\utils")

from players_season_builder import *

In [65]:
player_path = r"\Users\sebas\Desktop\UChicago - Q6\Sports Analytics\sports_analytics_project\data\Big Data - Player\1. Player"
file_type, sheet_name = "xlsx" , 0
data_source = "BDB-PLAYER"

player_season_data = combine_xl_data(player_path, file_type, sheet_name, data_source)

In [66]:
player_per_data = pd.read_excel(
    r"\Users\sebas\Desktop\UChicago - Q6\Sports Analytics\sports_analytics_project\data\Player Per Game PER Data.xlsx",
    sheet_name='Agg PER >24mins,NoPlayoffs')
player_per_data.columns = ['DATASET', 'DATE', 'PLAYER FULL NAME', 'OWN TEAM', 'MIN', 'PER']


In [67]:
# Filter Players 
df_player_filter = player_per_data.groupby(['DATASET', 'PLAYER FULL NAME'])['MIN'].agg(['mean', 'count'])
df_player_filter = df_player_filter.reset_index()
df_player_filter.columns
df_player_filter = df_player_filter.loc[ (df_player_filter['mean'] > 24) & (df_player_filter['count'] > 30) ]
# df_player_filter = df_player_filter[['DATASET','PLAYER FULL NAME']]
player_per_data = pd.merge(player_per_data, df_player_filter, on=['DATASET','PLAYER FULL NAME'])

# player_per_data[player_per_data['DATASET'].eq('2006-2007 Regular Season') & player_per_data['PLAYER FULL NAME'].eq('Luol Deng') ]


In [68]:
# Game Player
player_season_data['DATASET'] = player_season_data['DATASET'].str.replace(
    'NBA 2018-2019 Regular Season',
    '2018-2019 Regular Season'
)
player_season_data['DATASET'] = player_season_data['DATASET'].str.replace(
    'NBA 2019-2020 Regular Season',
    '2019-2020 Regular Season'
)
player_season_data['DATASET'] = player_season_data['DATASET'].str.replace(
    'NBA 2020 Playoffs', '2020 Playoffs')
player_season_data['DATASET'] = player_season_data['DATASET'].str.replace(
    'NBA 2019 Playoffs','2019 Playoffs')

player_season_data['DATE'] = pd.to_datetime(
    player_season_data['DATE']
    )

# Player PER Data
player_per_data['DATASET'] = player_per_data['DATASET'].str.replace(
    'NBA 2018-2019 Regular Season',
    '2018-2019 Regular Season'
)
player_per_data['DATASET'] = player_per_data['DATASET'].str.replace(
    'NBA 2019-2020 Regular Season',
    '2019-2020 Regular Season'
)

# Filter for Regular Season
player_regular_season = player_season_data[player_season_data['DATASET'].str.contains("Season")]
# Filter for playoffs
player_playoff = player_season_data[player_season_data['DATASET'].str.contains("Playoffs")]


In [69]:
player_regular_season['DATE'] = player_regular_season['DATE'].dt.strftime('%m/%d/%Y')
player_regular_season[['DATASET']] = player_regular_season[['DATASET']].astype('string')
player_per_data[['DATASET']] = player_per_data[['DATASET']].astype('string')

<ipython-input-69-004cabe60f09>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_regular_season['DATE'] = player_regular_season['DATE'].dt.strftime('%m/%d/%Y')
c:\Python39\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


Merge Player Info with Player PER Info w/ 24 min avg 


In [70]:
player_regular_season = pd.merge(
    player_regular_season,
    player_per_data,
    on=['DATASET', 'DATE', 'PLAYER FULL NAME','OWN TEAM', 'MIN']
)
# All MINS and own teams are the same

In [71]:
player_regular_season['DATE'] = pd.to_datetime(player_regular_season['DATE'])

In [72]:
test = list()
for season in player_regular_season['DATASET'].unique():
    for player in list(player_regular_season[player_regular_season['DATASET'] == season]['PLAYER FULL NAME'].unique()):
        temp_season = player_regular_season.loc[
            player_regular_season['DATASET'].eq(season) & player_regular_season['PLAYER FULL NAME'].eq(player)].sort_values(by=['PLAYER FULL NAME', 'DATE'])
        temp_season['DATE-DIFF'] = temp_season['DATE'].diff().dt.days
         # Not sure how to handle multiple Road games
        temp_season['RR VAL'] = temp_season['VENUE'].shift(1) + temp_season['VENUE']
        temp_season['RR SERIES'] = ( temp_season['VENUE'].shift(1) + temp_season['VENUE']) == "RR"
        n = temp_season['PER'].count()
        temp_season['AVG_PER'] = temp_season['PER'].mean()
        temp_season['OMIT_PER'] = (temp_season['AVG_PER']*n - temp_season['PER'])/(n-1)
       
        test.append(temp_season)


In [73]:
player_regular_season = pd.concat(test)
player_regular_season.loc[player_regular_season['DATE-DIFF'] > 14, ['DATE-DIFF']] = '14+'
player_regular_season.loc[player_regular_season['DATE-DIFF'].isna(), ['DATE-DIFF']] = 'Season Start'
player_regular_season['DATE-DIFF'] = player_regular_season['DATE-DIFF'].astype(str)
player_regular_season['DATE-DIFF'] = player_regular_season['DATE-DIFF']+" days"

In [74]:
player_regular_season.columns
# LA Lakers, LA Clippers
# New York, Brooklyn
player_regular_season['SAME CITY'] = 0
player_regular_season.loc[player_regular_season['OWN TEAM'].eq("LA Lakers") & player_regular_season['OPP TEAM'].eq("LA Clippers"), ['SAME CITY'] ] = 1
player_regular_season.loc[player_regular_season['OWN TEAM'].eq("LA Clippers") & player_regular_season['OPP TEAM'].eq("LA Lakers"),['SAME CITY']  ] = 1
player_regular_season.loc[player_regular_season['OWN TEAM'].eq("New York") & player_regular_season['OPP TEAM'].eq("Brooklyn"), ['SAME CITY'] ] = 1
player_regular_season.loc[player_regular_season['OWN TEAM'].eq("Brooklyn") & player_regular_season['OPP TEAM'].eq("New York"),['SAME CITY'] ] = 1

player_regular_season['TRAVEL'] = 1
# Played back to back home games
player_regular_season.loc[player_regular_season['RR VAL'].eq('HH'), ['TRAVEL']] = 0
# Last game was H, Current is R but in the Same City
player_regular_season.loc[player_regular_season['RR VAL'].eq('HR') & player_regular_season['SAME CITY'].eq(1), ['TRAVEL']] = 0
# Last game was R in Same City, Current game is H
player_regular_season.loc[player_regular_season['RR VAL'].eq('RH') & player_regular_season['SAME CITY'].shift(1).eq(1), ['TRAVEL']] = 0


In [75]:
player_regular_season_dummies = pd.concat([
    player_regular_season,
    pd.get_dummies(player_regular_season['DATE-DIFF']),
    pd.get_dummies(player_regular_season['VENUE'])
], axis=1)

In [76]:
player_season_avg = player_regular_season.groupby(['DATASET','PLAYER FULL NAME']).mean().sort_values(['DATASET','PLAYER FULL NAME'])
player_season_var= player_regular_season.groupby(['DATASET','PLAYER FULL NAME']).var().sort_values(['DATASET','PLAYER FULL NAME'])
player_season_sd = player_regular_season.groupby(['DATASET','PLAYER FULL NAME']).std().sort_values(['DATASET','PLAYER FULL NAME'])
player_season_HR= player_regular_season.groupby(['DATASET','PLAYER FULL NAME','VENUE']).mean().sort_values(['DATASET','PLAYER FULL NAME'])

In [77]:
# Let Andre do minute in the last 3 day calculations

# df

In [78]:
with pd.ExcelWriter("../data/BDB_Player.xlsx") as writer:
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    player_regular_season_dummies.to_excel(writer, sheet_name="General RS", index=False)
    df_player_filter.to_excel(writer, sheet_name="Players - Starters")
    player_season_avg.to_excel(writer, sheet_name="RS - AVG", index=True)
    player_season_HR.to_excel(writer, sheet_name="RS - HomeAway", index=True)